In [146]:
from stock_prediction.data_prep import Data_Prep
from stock_prediction.features_exo import exo_selection

In [147]:
import numpy as np 
import pandas as pd 
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error as MAPE
import pdb

In [148]:
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

In [149]:
stock = Data_Prep('safran', 44)

In [150]:
data_global = stock.data_prep()

In [151]:
data_exo = stock.select_features(data_global, Return = True, Log_Return=False, High_Low=False, High_Close=True, Low_Close=True,
                        Volume_Change=False, Period_Volum=False, Annual_Vol=False,
                        Period_Vol=False, Return_Index=False, Volum_Index=False, Relative_Return=False)


In [152]:
data_exo = data_exo[-258:]
data_exo.shape

(258, 4)

In [153]:
data_exo = stock.Price_Rebase(data_exo)

In [154]:
data_exo.columns

Index(['Date', 'High-Close_SAF.PA', 'Low-Close_SAF.PA', 'Return_SAF.PA_R'], dtype='object')

In [155]:
y_endo = data_exo['Return_SAF.PA_R']
y_exo = np.array(data_exo[['High-Close_SAF.PA', 'Low-Close_SAF.PA']])

In [156]:
#empty lists to store results 
list_y_pred = []
baseline = []
real_value = []
y_before = []
y_conf_low = []
y_conf_high = []
multi_days_results = []

# our parameter 
order = (0,1,0)
max_train = 44
splits = len(y_endo) - max_train

# we create all the index for slices covering all the TS
# only the first part of max_train won't have values 
folds = TimeSeriesSplit(n_splits=splits, max_train_size=max_train, test_size=1)

for (train_idx, test_idx) in folds.split(y_endo) :
   
    # we retrieve the real data in y 
    #corresponding of the indexes splited 
    y_train = y_endo[train_idx]
    y_exo_train = y_exo[train_idx]
    y_test = y_endo[test_idx]
    y_exo_test = y_exo[test_idx]
    y_true = y_endo[test_idx]
    base = y_train.iloc[-1]

 
    # fit our model on y_train of the split n°X
    model = ARIMA(endog=y_train, exog=y_exo_train, order=order).fit()
    
    # we find our y_pred for this slice on that part of the TS
    y_pred, std_pred, conf = model.forecast(steps=len(y_test), exog=y_exo_test, alpha=0.05)
    #pdb.set_trace()
    # we store the low price of confidence 
    y_conf_low.append(conf[0][0])
    # high price of confidence 
    y_conf_high.append(conf[0][1])
    # we store y_pred
    list_y_pred.append(y_pred[0])
    # we store the value for basescore
    baseline.append(base)
    # and the rel value to compare 
    real_value.append(y_true.values[0])
    y_before.append(y_train.iloc[-1])
    
    


# once we have all values we can compute MAPE and basescore   
base_score = MAPE(baseline, real_value)
mape_metric = MAPE(list_y_pred, real_value)

# we now need to store the results found to be able to work on them

multi_days_results = np.array([y_before, list_y_pred, y_conf_low, y_conf_high, real_value])
multi_days_results_df = pd.DataFrame({'yesterday' : y_before,
                                   'prediction' : list_y_pred,
                                   'conf_low' : y_conf_low,
                                   'conf_high' : y_conf_high,
                                   'true' : real_value,
                                      'date' : data_exo['Date'][-splits :]})
#                                     index=[data_exo.iloc[-splits :,'Date']]) #add dates 

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

...

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

In [157]:
std_pred[0]

1.2479899890368402

In [158]:
conf

array([[122.84857549, 127.74060636]])

In [159]:
base_score

0.016877071472693152

In [160]:
mape_metric

0.010885640082007953

In [161]:
multi_days_results_df

,yesterday,prediction,conf_low,conf_high,true,date
44,93.590265,96.096174,92.282196,99.910152,98.296147,2020-08-05
45,98.296147,99.864205,96.234503,103.493908,99.939148,2020-08-06
46,99.939148,100.543544,96.915254,104.171835,99.107510,2020-08-07
47,99.107510,100.787526,97.136512,104.438540,102.180531,2020-08-10
48,102.180531,104.650544,100.988607,108.312482,107.910752,2020-08-11
...,...,...,...,...,...,...
253,125.882359,125.108879,122.650715,127.567043,124.528784,2021-05-31
254,124.528784,125.972687,123.511217,128.434158,126.645653,2021-06-01
255,126.645653,128.142684,125.677116,130.608252,127.907629,2021-06-02
256,127.907629,126.450749,124.000085,128.901414,126.055371,2021-06-03


In [162]:
#perf_pred = []
#perf_true = []

# pred / before -1
perf_pred = ((multi_days_results[1, :] / multi_days_results[0, :]) -1)

# true / before - 1
perf_true = ((multi_days_results[4, :] / multi_days_results[0 , :]) -1)

# direction de pred 
dir_pred = perf_pred > 0
# direction de true
dir_true = perf_true > 0
# accurate direction 
dir_acc = dir_pred == dir_true

# perf of Low conf 
perf_low = (multi_days_results[2, :] / multi_days_results[0, :] -1)
# perf high conf
perf_high = (multi_days_results[3, :] / multi_days_results[0, :] -1)

# we try to analyze the confidence 
conf_ana = perf_high - (-perf_low)
dir_conf_low = perf_low > 0
dir_conf_high = perf_high > 0
conf_confirm = dir_conf_low == dir_conf_high



In [163]:
dir_acc.mean()

0.8271028037383178

In [164]:
conf_confirm.mean()

0.13551401869158877

In [165]:
multi_days_results_df['perf_pred'] = perf_pred
multi_days_results_df['perf_true'] = perf_true
multi_days_results_df['perf_low'] = perf_low
multi_days_results_df['perf_high'] = perf_high

In [166]:
multi_days_results_df

,yesterday,prediction,conf_low,conf_high,true,date,perf_pred,perf_true,perf_low,perf_high
44,93.590265,96.096174,92.282196,99.910152,98.296147,2020-08-05,0.026775,0.050282,-0.013977,0.067527
45,98.296147,99.864205,96.234503,103.493908,99.939148,2020-08-06,0.015952,0.016715,-0.020974,0.052879
46,99.939148,100.543544,96.915254,104.171835,99.107510,2020-08-07,0.006048,-0.008321,-0.030257,0.042353
47,99.107510,100.787526,97.136512,104.438540,102.180531,2020-08-10,0.016951,0.031007,-0.019887,0.053790
48,102.180531,104.650544,100.988607,108.312482,107.910752,2020-08-11,0.024173,0.056079,-0.011665,0.060011
...,...,...,...,...,...,...,...,...,...,...
253,125.882359,125.108879,122.650715,127.567043,124.528784,2021-05-31,-0.006144,-0.010753,-0.025672,0.013383
254,124.528784,125.972687,123.511217,128.434158,126.645653,2021-06-01,0.011595,0.016999,-0.008171,0.031361
255,126.645653,128.142684,125.677116,130.608252,127.907629,2021-06-02,0.011821,0.009965,-0.007648,0.031289
256,127.907629,126.450749,124.000085,128.901414,126.055371,2021-06-03,-0.011390,-0.014481,-0.030550,0.007770
